In [ ]:
import json

with open("./puzzles/puzzles_dict.json", "r") as f:
    puzzles_ds = json.load(f)

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def add_border(grid, padding):
  for i in range(padding):

    grid[i] = 0
    grid[-i] = 0
    grid[:, i] = 0
    grid[:, -i] = 0
  return grid

In [ ]:
def make_board(size):
  grid = np.ones((900, 900))
  grid = add_border(grid, 16)
  square_size = 900 // size

  for i in range(1, size):
    dim = i*square_size
    thickness = 2 if size < 7 else 1
    for j in range(-thickness, 2):
      cur = dim+j
      grid[cur] = 0
      grid[-cur] = 0
      grid[:, cur] = 0
      grid[:, -cur] = 0

  return grid

In [ ]:
def add_side_wall(grid, size, row, col, side):
  square_size = 900 // size
  top_left = (row*square_size, col*square_size)
  bottom_right = ((row+1)*square_size, (col+1)*square_size)

  if side == "left":

    if row == 0:
      for j in range(-8, 8):
        grid[top_left[0]:bottom_right[0]+8, top_left[1]+j] = 0
    elif row == size-1:
      for j in range(-8, 8):
        grid[top_left[0]-8:bottom_right[0], top_left[1]+j] = 0
    else:
      for j in range(-8, 8):
        grid[top_left[0]-8:bottom_right[0]+8, top_left[1]+j] = 0


  elif side == "right":

    if row == 0:
      for j in range(-8, 8):
        grid[top_left[0]:bottom_right[0]+8, bottom_right[1]+j] = 0
    elif row == size-1:
      for j in range(-8, 8):
        grid[top_left[0]-8:bottom_right[0], bottom_right[1]+j] = 0
    else:
      for j in range(-8, 8):
        grid[top_left[0]-8:bottom_right[0]+8, bottom_right[1]+j] = 0


  elif side == "top":

    if col == 0:
      for j in range(-8, 8):
        grid[top_left[0]+j, top_left[1]:bottom_right[1]+8] = 0
    elif col == size-1:
      for j in range(-8, 8):
        grid[top_left[0]+j, top_left[1]-8:bottom_right[1]] = 0
    else:
      for j in range(-8, 8):
        grid[top_left[0]+j, top_left[1]-8:bottom_right[1]+8] = 0


  else:

    if col == 0:
      for j in range(-8, 8):
        grid[bottom_right[0]+j, top_left[1]:bottom_right[1]+8] = 0
    elif col == size-1:
      for j in range(-8, 8):
        grid[bottom_right[0]+j, top_left[1]-8:bottom_right[1]] = 0
    else:
      for j in range(-8, 8):
        grid[bottom_right[0]+j, top_left[1]-8:bottom_right[1]+8] = 0

  return grid

In [ ]:
def draw_cage(grid, size, cage):
  for cell in cage["cells"]:
    row, col = cell


    if [row, col+1] not in cage["cells"] and col+1<size:
      grid = add_side_wall(grid, size, row, col, "right")
    if [row, col-1] not in cage["cells"] and col-1>=0:
      grid = add_side_wall(grid, size, row, col, "left")
    if [row+1, col] not in cage["cells"] and row+1<size:
      grid = add_side_wall(grid, size, row, col, "bottom")
    if [row-1, col] not in cage["cells"] and row-1>=0:
      grid = add_side_wall(grid, size, row, col, "top")

  return grid

In [ ]:
df = pd.read_csv("./symbols/TMNIST_NotoSans.csv")

In [ ]:
noto_sans = df[df['names'].str.contains('notosans', case=False, na=False)]
noto_sans = noto_sans[noto_sans['names'].str.contains('NotoSans-Regular', case=False, na=False)]

In [ ]:
noto_sans = noto_sans.sort_values(by='labels')

image_arrays = [
    (255 - row.values.reshape(28, 28)) / 255.0
    for _, row in noto_sans.drop(columns=['names','labels']).iterrows()
]

labels = noto_sans['labels'].tolist()

In [ ]:
def normalize_symbol(file_path):

  im = Image.open("./symbols/operators/" + file_path).convert("RGBA")

  rgba = np.array(im)
  r, g, b, a = rgba[:, :, 0], rgba[:, :, 1], rgba[:, :, 2], rgba[:, :, 3]
  gray = 0.299 * r + 0.587 * g + 0.114 * b
  alpha = a / 255.0
  composited = gray * alpha + 255 * (1 - alpha)

  normalized = composited / 255.0
  return normalized

In [ ]:
def insert_number_in_cage(grid, size, cell, number, position):


  square_size = 900 // size

  number_size = square_size // 4

  pil_img = Image.fromarray(image_arrays[number])

  resized_img = pil_img.resize((number_size, number_size), resample=Image.NEAREST)
  cropped = np.array(resized_img.crop((resized_img.width // 5, 0, resized_img.width * 4 // 5, resized_img.height)))

  height, width = cropped.shape
  col_position = 20 + position*width


  top_left = (cell[0]*square_size + 20, cell[1]*square_size + col_position)
  grid[top_left[0]:top_left[0]+height, top_left[1]:top_left[1]+width] = cropped
  return grid

In [ ]:
def insert_symbol_in_cage(grid, size, cell, symbol, position):

  square_size = 900 // size

  number_size = square_size // 4

  symbol = Image.fromarray(normalize_symbol(symbol + ".png"))


  resized_img = symbol.resize((number_size, number_size), resample=Image.NEAREST)
  cropped = np.array(resized_img.crop((resized_img.width // 6, 0, resized_img.width * 5 // 6, resized_img.height)))

  height, width = cropped.shape
  col_position = 20 + position*width

  top_left = (cell[0]*square_size + 20, cell[1]*square_size + col_position)
  grid[top_left[0]:top_left[0]+height, top_left[1]:top_left[1]+width] = cropped
  return grid

In [ ]:
def write_to_cage(board, size, cage):
  start_cell = cage["cells"][0]

  if len(cage["cells"]) > 1:
    for cell in cage["cells"][1:]:
      if cell[0]<start_cell[0]:
        start_cell = cell
      elif cell[0]==start_cell[0] and cell[1]<start_cell[1]:
        start_cell = cell

  digits = []
  temp = cage["target"]
  while temp > 0:
      digits.append(int(temp % 10))
      temp //= 10
  digits = digits[::-1]

  for i, digit in enumerate(digits):
    board = insert_number_in_cage(board, size, start_cell, digit, i)

  if cage["op"] != "":
    board = insert_symbol_in_cage(board, size, start_cell, cage["op"], len(digits))

  return board


In [ ]:
def make_board_full(size, puzzle):
  board = make_board(size)
  for cage in puzzle:
    board = draw_cage(board, size, cage)
    board = write_to_cage(board, size, cage)
  return board

In [ ]:
def make_and_save(size, puzzle, iter):
  board = make_board_full(size, puzzle)
  array_uint8 = (board * 255).astype(np.uint8)
  image = Image.fromarray(array_uint8, mode='L')
  image.save('./board_images/board'+str(size)+'_'+str(iter)+'.png')

In [ ]:
for i in range(30):
  make_and_save(7, puzzles_ds["7"][i], i)

In [ ]:
for size in range(3, 7):
  for i in range(len(puzzles_ds[str(size)])):
    make_and_save(size, puzzles_ds[str(size)][i], i)